In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
import numpy as np

In [6]:
# Trying to import a version of tensorflow that may not exist outside CoLab
try:
  # %tensorflow_version only exists in CoLab
  % %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf

In [0]:
# Tensorflow Hub is a library of reusable, pretrained ML components
import tensorflow_hub as hub

# Many datasets, such as the IMDb dataset, are built into TF
import tensorflow_datasets as tfds

In [10]:
# Environment description
print("Version: ", tf.__version__)

# eager execution - operations return values rather than graphs to build comp models later
print("Eager Mode: ", tf.executing_eagerly()) 

print("Hub version: ", hub.__version__)
print("GPU is ", "available"  if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.0.0-rc1
Eager Mode:  True
Hub version:  0.6.0
GPU is  NOT AVAILABLE


## Downloading Dataset

In [18]:
train_valiation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name='imdb_reviews',
    split=(train_validation_split, tfds.Split.TEST), 
    as_supervised=True
)

    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(), dtype=tf.string),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


In [0]:
# Big wall of errors?

In [0]:
# Extracting features and labels for the first 10 training samples
# Each sample is a sentence representing a review
# Each label is a boolean flag 1 or 0 representing pos/neg reviews
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [23]:
train_examples_batch

<tf.Tensor: id=449, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [25]:
# Printing the first 10 labels
train_labels_batch

<tf.Tensor: id=460, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

## Building the Model

### Why use Pre-Trained Embeddings?
- Embedding layers are used to transform sequences of words in text data into word vectors in an `n` dimensional space.
- This vectorization of words allows NNs to compute numerical relationships between words in the `n` dimensional space e.g. how "close" two words are, or the "translation" vector from one word to another.
- Word embeddings can be learnt as part of the training process, but this can be problematic
  - Not enough data
  - Computation cost
- Advantages of using **pre-trained word embeddings**
  - Don't have to worry about text preprocessing
  - Can benefit from transfer learning: use pretrained and prelearnt relations between words rather than learning them yourself.
  - Embedding has a fixed size, so its easier to process.

In [0]:
# URL for the embedding layer stored on tfhub's CDN
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'

In [0]:
# Instantiating an Embedding layer with the embedding URL
hub_layer = hub.KerasLayer(embedding,
                          input_shape=[], # can be any shape
                          dtype=tf.string, 
                          trainable=True)

In [30]:
# This Embedding layer will successfully transform a sequence of text reviews into numerical tensors
# Because the word embedding layer was 20 dimensional, we will have transformed the first three reviews
# into 20-dimensional tensors
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=650, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [33]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [34]:
# Model layers - names, params, OP shapes
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


The three layers in the model are as follows
- **Embedding**: `TensorflowHub` layer that uses pre-trained weights to map a sentence (of variable length) to a 20-dimensional embedding vector. It splits the sentence into tokens (individual words), embeds each token (so each word is also mapped to an `n`-dimensional vector) and then combines the embedding. The dimension is (`num_examples`, `embedding_dimension`).
- `dense_2`: Fixed length OP vectors from the first layer are piped through a fully-connected layer with 16 units.
- `dense_3`: Single output node because binary classification problem. `sigmoid` activation function because it has to predict a probability (from 0 to 1) that the given sample belongs to the positive class.

In [0]:
# Compiling the model
model.compile(optimizer='adam', 
             loss='binary_crossentropy', 
             metrics=['accuracy'])

## Training the Model

In [36]:
history = model.fit(train_data.shuffle(10000).batch(512), 
                   epochs=20, 
                   validation_data=validation_data.batch(512), 
                   verbose=1)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================] - 7s 233ms/step - loss: 1.8476 - accuracy: 0.5018 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 6s 200ms/step - loss: 0.8312 - accuracy: 0.5345 - val_loss: 0.7080 - val_accuracy: 0.5383
Epoch 3/20
30/30 [==============================] - 6s 209ms/step - loss: 0.7006 - accuracy: 0.5521 - val_loss: 0.6867 - val_accuracy: 0.5730
Epoch 4/20
30/30 [==============================] - 6s 202ms/step - loss: 0.6787 - accuracy: 0.5894 - val_loss: 0.6694 - val_accuracy: 0.6000
Epoch 5/20
30/30 [==============================] - 6s 200ms/step - loss: 0.6610 - accuracy: 0.6146 - val_loss: 0.6528 - val_accuracy: 0.6223
Epoch 6/20
30/30 [==============================] - 6s 211ms/step - loss: 0.6427 - accuracy: 0.6415 - val_loss: 0.6362 - val_accuracy: 0.6434
Epoch 7/20
30/30 [==============================] - 6s 207ms/step - loss: 0.6248 - accuracy: 0.6657 - val_loss: 0.6199 - val_accuracy: 0.6634
Epoch 8/2

## Model Evaluation

In [37]:
results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" %(name, value))

loss: 0.337
accuracy: 0.856


## My Comments
- Using the pre-trained word embeddings did not seem to have a substantial effect on the accuracy in this case. In fact, the performance actually deteriorated.
- For comparison, the IMDb model trained on 40 epochs had a test set accuracy of 86.93%, which is actually higher than the accuracy with the word embeddings. That was despite overfitting.
- Too large a dataset so random shuffling should not have caused this issue.
- Batch size was the same too.
- Maybe the word embeddings should have been higher dimensional?
- However, lets contextualize this: for the original IMDb model, I used a 10k dimensional vector for each word of the sentence. In this case, I am using a 20 dimensional vector for **each review**.
  - I also didn't have to do any preprocessing (debatable, since the setup was equivalent to preprocessing itself)
  - Computational complexity is several orders of magnitude lower (20 dim vs 10k * `n_word` dim)
  - Computation time comparison isn't fair. Online is not the same as offline. 
  - Still, 203 ms per step vs ~90 mus per sample.
- I guess maybe a higher dimensional word embedding could be useful?
- Stanford's GloVe embedding gave a much higher performance, though, so this doesn't mean word embeddings are useless.
- Also, we're using a word embedding trained with **news** to do sentiment analysis on **reviews**. There's obviously going to be some loss in accuracy.
  - F. Chollet said that word embeddings for one application in one language are usually not easily transferable to another application, even in the same language.
  - With that in mind, the performance is still pretty good.